<a href="https://colab.research.google.com/github/gaydever228/bioinformatics/blob/main/task20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Designed by **Sergei Kozlov** M05-315 in *Dolgoprudnii, Russia*

# Implement ConvolutionCyclopeptideSequencing

In [ ]:
task_file = open('rosalind_ba4i.txt','r')
task = ''.join(task_file).split()
m = int(task[0])
n = int(task[1])
spectrum = [int(i) for i in task[2:]]

In [ ]:
def expand(peptides, mtop):
    expands = []
    for p in peptides:
        for m in mtop:
            expands.append(p + [m])
    return expands

def lspectrum(peptide):
    m = [0]
    leng = len(peptide)
    for i in range(leng):
        mp = m[i] + peptide[i]
        m.append(mp)
    spectrum = [0]
    for i in range(leng):
        for j in range(i+1, leng+1):
            spectrum.append(m[j] - m[i])
    return sorted(spectrum)

def cyclospectrum(peptide):
    cyclospectrum = [0, sum(peptide)]
    pp = peptide*2
    for i in range(1, len(peptide)):
        for j in range(len(peptide)):
            p = pp[j:j + i]
            cyclospectrum.append(sum(p))
    return sorted(cyclospectrum)

def lscore(peptide, spectrum):
    score = 0
    sspectrum = spectrum.copy()
    for i in lspectrum(peptide):
        if i in sspectrum:
            score += 1
            sspectrum.remove(i)
    return score

def cscore(peptide, spectrum):
    score = 0
    for i in set(cyclospectrum(peptide)+spectrum):
            score += min(cyclospectrum(peptide).count(i), spectrum.count(i))
    return score

def trim(leaderboard, spectrum, n):
    scores = {}
    if len(leaderboard) <= n:
        return leaderboard
    for i, p in enumerate(leaderboard):
        scores[i] = lscore(p, spectrum)
    sscores = sorted(scores.values())[::-1]
    trim = sscores[n-1]
    tleaderboard = [leaderboard[i] for i, score in scores.items() if score >= trim]
    return tleaderboard

In [ ]:
def leaderboard(n, spectrum, mtop):
    score = 0
    leaderboard = [[]]
    lpeptide = []
    res = set()
    while leaderboard:
        leaderboard = expand(leaderboard, mtop)
        for p in leaderboard:
            if sum(p) == spectrum[-1]:
                if cscore(p, spectrum) > score:
                    score = cscore(p, spectrum)
                    lpeptide = p
            elif sum(p) > spectrum[-1]:
                leaderboard = list(filter(lambda i: i != p, leaderboard))
        leaderboard = trim(leaderboard, spectrum, n)
    res = "".join(['-'+str(i) for i in lpeptide])
    return res

def conv(spectrum):
    convolution = {}
    mass = []
    sspectrum = sorted(spectrum)
    for i in range(len(sspectrum)-1):
        for j in range(i, len(sspectrum)):
            mass.append(sspectrum[j]-sspectrum[i])
    for m in mass:
        if (m >= 57 and m <= 200):
            convolution[m] = mass.count(m)
    return convolution

def top(conv, m):
    sconv = sorted(conv.items(), key=lambda m: m[1])[::-1]
    mtop = [mass for mass, k in sconv if k >= sconv[m][1]]
    return mtop

def convCycloSequencing(m, n, spectrum):
    mtop = top(conv(spectrum), m)
    lpeptide = leaderboard(n, spectrum, mtop)
    return lpeptide[1:]

In [ ]:
convCycloSequencing(m, n, spectrum)

'128-57-128-99-156-71-137-129-128-87-114-114-186'